In [28]:
# 📊 COMPREHENSIVE API CALL SUCCESS DIAGNOSTIC - ALL ENDPOINTS
import pandas as pd
import psycopg2
import json
import sys
from datetime import datetime

# Add config path and load endpoint configuration
sys.path.append('../endpoints/config')
import nba_endpoints_config as config

print("🔍 NBA API SUCCESS DIAGNOSTIC - ALL ENDPOINTS")
print("="*80)
print(f"⏰ Diagnostic run at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🏷️  Using NEW table naming convention: nba_{endpoint}_{dataframe_name}")
print("   e.g., nba_boxscoretraditionalv3_playerstats, nba_boxscoretraditionalv3_teamstats")
print()

# Connect to database
with open('../endpoints/config/database_config.json', 'r') as f:
    db_config = json.load(f)

conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],  # Note: config uses 'name' instead of 'database'
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

# Get all existing tables
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' 
    ORDER BY table_name
""")
all_tables = [row[0] for row in cursor.fetchall()]

# Master table counts - using correct column names from database
master_tables = {
    'nba_games': {'id_column': 'gameid', 'count': 0},
    'nba_players': {'id_column': 'playerid', 'count': 0},  # Correct: playerid
    'nba_teams': {'id_column': 'teamid', 'count': 0}       # Correct: teamid
}

# Get master table counts
for master_table, info in master_tables.items():
    if master_table in all_tables:
        cursor.execute(f"SELECT COUNT(DISTINCT {info['id_column']}) FROM {master_table}")
        info['count'] = cursor.fetchone()[0]

print("📋 MASTER TABLE COUNTS:")
for table, info in master_tables.items():
    print(f"   {table}: {info['count']:,} unique {info['id_column']}s")

# Get failed API calls by endpoint
try:
    cursor.execute("""
        SELECT endpoint_prefix, 
               COUNT(*) as total_failed,
               COUNT(CASE WHEN gameid IS NOT NULL THEN 1 END) as failed_games,
               COUNT(CASE WHEN playerid IS NOT NULL THEN 1 END) as failed_players,
               COUNT(CASE WHEN teamid IS NOT NULL THEN 1 END) as failed_teams
        FROM failed_api_calls 
        GROUP BY endpoint_prefix
        ORDER BY endpoint_prefix
    """)
    failed_calls = {}
    for row in cursor.fetchall():
        failed_calls[row[0]] = {
            'total_failed': row[1],
            'failed_games': row[2],
            'failed_players': row[3], 
            'failed_teams': row[4]
        }
except Exception as e:
    print(f"⚠️  Could not read failed_api_calls table: {e}")
    failed_calls = {}

print(f"\n🚫 FAILED API CALLS BY ENDPOINT:")
if failed_calls:
    for endpoint, counts in failed_calls.items():
        total_failed = counts.get('total_failed', 0)
        print(f"   {endpoint}: {total_failed:,} failed calls")
else:
    print("   No failed API calls recorded")

# Analyze all endpoints
print(f"\n📊 ENDPOINT SUCCESS ANALYSIS:")
print("-" * 80)

endpoint_stats = []
total_successful_calls = 0
total_possible_calls = 0

for category_name, endpoints in config.ALL_ENDPOINTS.items():
    print(f"\n🎯 {category_name.upper()} ENDPOINTS:")
    
    for endpoint_config in endpoints:
        endpoint_name = endpoint_config['endpoint']
        
        # Determine master table and ID column based on endpoint parameters
        params = endpoint_config.get('parameters', {})
        if 'game_id' in params or 'from_mastergames' in str(params):
            master_table = 'nba_games'
            endpoint_id_column = 'gameid'  # Column name in endpoint tables
            total_ids = master_tables['nba_games']['count']
        elif 'player_id' in params or 'from_masterplayers' in str(params):
            master_table = 'nba_players' 
            endpoint_id_column = 'playerid'  # Column name in endpoint tables
            total_ids = master_tables['nba_players']['count']
        elif 'team_id' in params or 'from_masterteams' in str(params):
            master_table = 'nba_teams'
            endpoint_id_column = 'teamid'  # Column name in endpoint tables
            total_ids = master_tables['nba_teams']['count']
        else:
            # League-level endpoints don't have individual ID tracking
            print(f"   📈 {endpoint_name}: League-level endpoint (no ID tracking)")
            continue
        
        # Find tables for this endpoint (using new naming convention)
        # New format: nba_{endpoint_name}_{dataframe_name} 
        # e.g., nba_boxscoretraditionalv3_playerstats, nba_boxscoretraditionalv3_teamstats
        endpoint_tables = [t for t in all_tables if t.startswith(f'nba_{endpoint_name.lower()}_')]
        
        if not endpoint_tables:
            successful_ids = 0
            print(f"   ❌ {endpoint_name}: No tables found (0 successful calls)")
        else:
            # Show found tables for debugging (first run)
            if len(endpoint_tables) == 1:
                print(f"   📊 {endpoint_name}: Found 1 table: {endpoint_tables[0]}")
            else:
                print(f"   📊 {endpoint_name}: Found {len(endpoint_tables)} tables: {endpoint_tables[0]}...")
            
            # Use first table (alphabetically) to count unique IDs processed
            first_table = sorted(endpoint_tables)[0]
            try:
                cursor.execute(f"SELECT COUNT(DISTINCT {endpoint_id_column}) FROM {first_table}")
                successful_ids = cursor.fetchone()[0]
            except Exception as e:
                # Try alternative column names if the primary fails
                try:
                    if 'personid' in str(e).lower() and endpoint_id_column == 'playerid':
                        cursor.execute(f"SELECT COUNT(DISTINCT personid) FROM {first_table}")
                        successful_ids = cursor.fetchone()[0]
                    elif endpoint_id_column == 'playerid':
                        # Try other common player ID column names
                        cursor.execute(f"SELECT COUNT(DISTINCT personid) FROM {first_table}")
                        successful_ids = cursor.fetchone()[0]
                    else:
                        # Fallback to row count
                        cursor.execute(f"SELECT COUNT(*) FROM {first_table}")
                        successful_ids = cursor.fetchone()[0]
                        print(f"   ⚠️  {endpoint_name}: Using row count instead of unique ID count")
                except:
                    successful_ids = 0
                    print(f"   ⚠️  {endpoint_name}: Error reading {first_table} - {str(e)[:50]}...")
                    continue
        
        # Get failed calls for this endpoint (using new naming convention)
        endpoint_prefix = f"nba_{endpoint_name.lower()}"  # This should match how processor stores failed calls
        failed_ids = 0
        if endpoint_prefix in failed_calls:
            if master_table == 'nba_games':
                failed_ids = failed_calls[endpoint_prefix]['failed_games']
            elif master_table == 'nba_players':
                failed_ids = failed_calls[endpoint_prefix]['failed_players'] 
            elif master_table == 'nba_teams':
                failed_ids = failed_calls[endpoint_prefix]['failed_teams']
        
        # Calculate success metrics
        processed_ids = successful_ids + failed_ids
        remaining_ids = max(0, total_ids - processed_ids)
        success_rate = (successful_ids / total_ids * 100) if total_ids > 0 else 0
        
        # Display results
        status_icon = "✅" if success_rate > 90 else "🟡" if success_rate > 50 else "🔴" if success_rate > 0 else "❌"
        print(f"   {status_icon} {endpoint_name}: {successful_ids:,}/{total_ids:,} successful ({success_rate:.1f}%)")
        
        # Track totals
        endpoint_stats.append({
            'endpoint': endpoint_name,
            'category': category_name,
            'master_table': master_table,
            'total_ids': total_ids,
            'successful_ids': successful_ids,
            'failed_ids': failed_ids,
            'remaining_ids': remaining_ids,
            'success_rate': success_rate
        })
        
        total_successful_calls += successful_ids
        total_possible_calls += total_ids

# Summary statistics
print(f"\n{'='*80}")
print("📊 OVERALL SUCCESS SUMMARY")
print(f"{'='*80}")

if endpoint_stats:
    avg_success_rate = sum(stat['success_rate'] for stat in endpoint_stats) / len(endpoint_stats)
    completed_endpoints = sum(1 for stat in endpoint_stats if stat['success_rate'] > 95)
    partial_endpoints = sum(1 for stat in endpoint_stats if 0 < stat['success_rate'] <= 95)
    empty_endpoints = sum(1 for stat in endpoint_stats if stat['success_rate'] == 0)
    
    print(f"🎯 Total endpoints analyzed: {len(endpoint_stats)}")
    print(f"✅ Completed endpoints (>95%): {completed_endpoints}")
    print(f"🟡 Partial endpoints (1-95%): {partial_endpoints}")
    print(f"❌ Empty endpoints (0%): {empty_endpoints}")
    print(f"📈 Average success rate: {avg_success_rate:.1f}%")
    print(f"🔢 Total successful API calls: {total_successful_calls:,}")
    
    # Show top and bottom performers
    sorted_stats = sorted(endpoint_stats, key=lambda x: x['success_rate'], reverse=True)
    
    print(f"\n🏆 TOP 5 PERFORMING ENDPOINTS:")
    for stat in sorted_stats[:5]:
        print(f"   ✅ {stat['endpoint']}: {stat['success_rate']:.1f}% ({stat['successful_ids']:,}/{stat['total_ids']:,})")
    
    if len(sorted_stats) > 5:
        print(f"\n⚠️  BOTTOM 5 PERFORMING ENDPOINTS:")
        for stat in sorted_stats[-5:]:
            print(f"   {('❌' if stat['success_rate'] == 0 else '🔴')} {stat['endpoint']}: {stat['success_rate']:.1f}% ({stat['successful_ids']:,}/{stat['total_ids']:,})")

else:
    print("❌ No endpoint statistics available")

print(f"\n🔄 Run this cell again to refresh the diagnostic")
print(f"⚡ This diagnostic can run while the endpoint processor is actively collecting data")

cursor.close()
conn.close()

🔍 NBA API SUCCESS DIAGNOSTIC - ALL ENDPOINTS
⏰ Diagnostic run at: 2025-08-28 19:52:27
🏷️  Using NEW table naming convention: nba_{endpoint}_{dataframe_name}
   e.g., nba_boxscoretraditionalv3_playerstats, nba_boxscoretraditionalv3_teamstats

📋 MASTER TABLE COUNTS:
   nba_games: 52,782 unique gameids
   nba_players: 571 unique playerids
   nba_teams: 30 unique teamids
⚠️  Could not read failed_api_calls table: column "gameid" does not exist
LINE 4:                COUNT(CASE WHEN gameid IS NOT NULL THEN 1 END...
                                       ^


🚫 FAILED API CALLS BY ENDPOINT:
   No failed API calls recorded

📊 ENDPOINT SUCCESS ANALYSIS:
--------------------------------------------------------------------------------

🎯 GAME_BASED ENDPOINTS:
   📊 BoxScoreAdvancedV3: Found 4 tables: nba_boxscoreadvancedv3_0...
   ⚠️  BoxScoreAdvancedV3: Error reading nba_boxscoreadvancedv3_0 - current transaction is aborted, commands ignored u...
   ❌ BoxScoreAdvancedV2: No tables found (0 succes

In [23]:
# 🔍 QUICK TABLE STRUCTURE CHECK
import pandas as pd
import psycopg2
import json

# Connect to database
with open('../endpoints/config/database_config.json', 'r') as f:
    db_config = json.load(f)

conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

# Check master table structures
master_tables = ['nba_games', 'nba_players', 'nba_teams']

print("🔍 MASTER TABLE COLUMN STRUCTURES:")
print("="*50)

table_structures = {}
for table in master_tables:
    try:
        cursor.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = '{table}' 
            ORDER BY ordinal_position
        """)
        columns = cursor.fetchall()
        table_structures[table] = columns
        print(f"\n📋 {table.upper()}:")
        for col_name, col_type in columns[:8]:  # Show first 8 columns
            print(f"   {col_name} ({col_type})")
        if len(columns) > 8:
            print(f"   ... and {len(columns) - 8} more columns")
            
    except Exception as e:
        print(f"❌ Error checking {table}: {e}")

# Check if any endpoint tables exist to see their structure
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' 
    AND table_name LIKE 'nba_%'
    AND table_name NOT IN ('nba_games', 'nba_players', 'nba_teams')
    LIMIT 3
""")
sample_tables = [row[0] for row in cursor.fetchall()]

if sample_tables:
    print(f"\n🔍 SAMPLE ENDPOINT TABLE STRUCTURES:")
    print("="*50)
    
    for table in sample_tables:
        try:
            cursor.execute(f"""
                SELECT column_name, data_type 
                FROM information_schema.columns 
                WHERE table_name = '{table}' 
                ORDER BY ordinal_position
            """)
            columns = cursor.fetchall()
            print(f"\n📋 {table.upper()}:")
            for col_name, col_type in columns[:6]:  # Show first 6 columns
                print(f"   {col_name} ({col_type})")
            if len(columns) > 6:
                print(f"   ... and {len(columns) - 6} more columns")
                
        except Exception as e:
            print(f"❌ Error checking {table}: {e}")

cursor.close()
conn.close()

🔍 MASTER TABLE COLUMN STRUCTURES:

📋 NBA_GAMES:
   gameid (character varying)
   seasonid (character varying)
   gamedate (date)
   hometeamid (character varying)
   hometeamabbreviation (character varying)
   hometeamname (character varying)
   awayteamid (character varying)
   awayteamabbreviation (character varying)
   ... and 8 more columns

📋 NBA_PLAYERS:
   playerid (character varying)
   playername (character varying)
   teamid (character varying)
   teamabbreviation (character varying)
   season (character varying)
   position (character varying)
   height (character varying)
   weight (character varying)
   ... and 11 more columns

📋 NBA_TEAMS:
   teamid (character varying)
   teamname (character varying)
   teamabbreviation (character varying)
   teamcity (character varying)
   teamstate (character varying)
   yearfounded (integer)
   conference (character varying)
   division (character varying)
   ... and 3 more columns

🔍 SAMPLE ENDPOINT TABLE STRUCTURES:

📋 NBA_ENDPOINT_F

In [29]:
# 🔍 CHECK TABLE NAMING CONVENTION
import psycopg2
import json

with open('../endpoints/config/database_config.json', 'r') as f:
    db_config = json.load(f)

conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

# Get all NBA endpoint tables
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' 
    AND table_name LIKE 'nba_%'
    AND table_name NOT IN ('nba_games', 'nba_players', 'nba_teams')
    ORDER BY table_name
""")
endpoint_tables = [row[0] for row in cursor.fetchall()]

print("🏷️  TABLE NAMING ANALYSIS:")
print("="*60)

# Check for old naming convention (ends with _0, _1, etc.)
old_naming = [t for t in endpoint_tables if t.split('_')[-1].isdigit()]
new_naming = [t for t in endpoint_tables if not t.split('_')[-1].isdigit()]

print(f"📊 Total endpoint tables: {len(endpoint_tables)}")
print(f"🔴 Old naming convention (ends with _0, _1, etc.): {len(old_naming)}")
print(f"✅ New naming convention (meaningful names): {len(new_naming)}")

if old_naming:
    print(f"\n⚠️  TABLES USING OLD NAMING:")
    for table in old_naming[:10]:  # Show first 10
        print(f"   {table}")
    if len(old_naming) > 10:
        print(f"   ... and {len(old_naming) - 10} more")

if new_naming:
    print(f"\n✅ SAMPLE TABLES USING NEW NAMING:")
    for table in new_naming[:10]:  # Show first 10
        print(f"   {table}")
    if len(new_naming) > 10:
        print(f"   ... and {len(new_naming) - 10} more")

cursor.close()
conn.close()

🏷️  TABLE NAMING ANALYSIS:
📊 Total endpoint tables: 103
🔴 Old naming convention (ends with _0, _1, etc.): 51
✅ New naming convention (meaningful names): 52

⚠️  TABLES USING OLD NAMING:
   nba_boxscoreadvancedv3_0
   nba_boxscoreadvancedv3_1
   nba_boxscorefourfactorsv3_0
   nba_boxscorefourfactorsv3_1
   nba_boxscoremiscv3_0
   nba_boxscoremiscv3_1
   nba_boxscoreplayertrackv3_0
   nba_boxscoreplayertrackv3_1
   nba_boxscorescoringv3_0
   nba_boxscorescoringv3_1
   ... and 41 more

✅ SAMPLE TABLES USING NEW NAMING:
   nba_boxscoreadvancedv3_playerstats
   nba_boxscoreadvancedv3_teamstats
   nba_boxscorefourfactorsv3_playerstats
   nba_boxscorefourfactorsv3_teamstats
   nba_boxscoremiscv3_playerstats
   nba_boxscoremiscv3_teamstats
   nba_boxscoreplayertrackv3_playerstats
   nba_boxscoreplayertrackv3_teamstats
   nba_boxscorescoringv3_playerstats
   nba_boxscorescoringv3_teamstats
   ... and 42 more


In [30]:
# Check specifically for CommonPlayerInfo tables
commonplayerinfo_tables = [t for t in endpoint_tables if 'commonplayerinfo' in t.lower()]
print(f"\n🔍 COMMONPLAYERINFO TABLES:")
for table in commonplayerinfo_tables:
    print(f"   {table}")

# Check creation times if possible
if commonplayerinfo_tables:
    print(f"\n⏰ TABLE CREATION INVESTIGATION:")
    for table in commonplayerinfo_tables:
        try:
            cursor = conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM {table}")
            count = cursor.fetchone()[0]
            print(f"   {table}: {count:,} rows")
            cursor.close()
        except Exception as e:
            print(f"   {table}: Error reading - {e}")

# Reconnect since we closed it
conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)


🔍 COMMONPLAYERINFO TABLES:
   nba_commonplayerinfo_0
   nba_commonplayerinfo_1
   nba_commonplayerinfo_2
   nba_commonplayerinfo_availableseasons
   nba_commonplayerinfo_commonplayerinfo
   nba_commonplayerinfo_playerheadlinestats

⏰ TABLE CREATION INVESTIGATION:
   nba_commonplayerinfo_0: Error reading - connection already closed
   nba_commonplayerinfo_1: Error reading - connection already closed
   nba_commonplayerinfo_2: Error reading - connection already closed
   nba_commonplayerinfo_availableseasons: Error reading - connection already closed
   nba_commonplayerinfo_commonplayerinfo: Error reading - connection already closed
   nba_commonplayerinfo_playerheadlinestats: Error reading - connection already closed


In [32]:
# 🗑️  DELETE ALL TABLES WITH OLD NAMING CONVENTION
import psycopg2
import json

print("🗑️  DATABASE CLEANUP - REMOVING OLD NAMING CONVENTION TABLES")
print("="*70)

# Use the old_naming list we already have
print(f"📊 Found {len(old_naming)} tables with old naming convention to delete")
print(f"✅ Keeping {len(new_naming)} tables with new meaningful naming")

# Show what we're about to delete
print(f"\n⚠️  TABLES TO BE DELETED (first 10):")
for table in old_naming[:10]:
    print(f"   🔴 {table}")
if len(old_naming) > 10:
    print(f"   ... and {len(old_naming) - 10} more tables")

# Safety confirmation
execute_cleanup = input(f"\n❓ Are you sure you want to DELETE {len(old_naming)} tables? Type 'DELETE' to confirm: ")

if execute_cleanup == 'DELETE':
    print(f"\n🚀 Starting cleanup of {len(old_naming)} tables...")
    
    # Connect to database
    conn = psycopg2.connect(
        host=db_config['host'],
        database=db_config['name'],
        user=db_config['user'],
        password=db_config['password'],
        port=db_config['port']
    )
    cursor = conn.cursor()
    
    deleted_count = 0
    error_count = 0
    
    for i, table in enumerate(old_naming, 1):
        try:
            print(f"   🗑️  [{i:3d}/{len(old_naming)}] Deleting {table}...")
            cursor.execute(f"DROP TABLE IF EXISTS {table}")
            conn.commit()
            deleted_count += 1
            
            # Progress indicator every 10 tables
            if i % 10 == 0:
                print(f"      ✅ Progress: {i}/{len(old_naming)} ({i/len(old_naming)*100:.1f}%)")
                
        except Exception as e:
            print(f"      ❌ Error deleting {table}: {e}")
            error_count += 1
            continue
    
    cursor.close()
    conn.close()
    
    print(f"\n🎉 CLEANUP COMPLETE!")
    print(f"   ✅ Successfully deleted: {deleted_count} tables")
    print(f"   ❌ Errors encountered: {error_count} tables")
    print(f"   🏁 Database now uses only meaningful table names!")
    
    if deleted_count > 0:
        print(f"\n📈 BEFORE: {len(old_naming) + len(new_naming)} total tables ({len(old_naming)} old + {len(new_naming)} new)")
        print(f"📉 AFTER:  ~{len(new_naming)} total tables (all with meaningful names)")
        
else:
    print(f"\n❌ Cleanup cancelled. No tables were deleted.")
    print(f"   Type 'DELETE' exactly to confirm deletion next time.")

🗑️  DATABASE CLEANUP - REMOVING OLD NAMING CONVENTION TABLES
📊 Found 51 tables with old naming convention to delete
✅ Keeping 52 tables with new meaningful naming

⚠️  TABLES TO BE DELETED (first 10):
   🔴 nba_boxscoreadvancedv3_0
   🔴 nba_boxscoreadvancedv3_1
   🔴 nba_boxscorefourfactorsv3_0
   🔴 nba_boxscorefourfactorsv3_1
   🔴 nba_boxscoremiscv3_0
   🔴 nba_boxscoremiscv3_1
   🔴 nba_boxscoreplayertrackv3_0
   🔴 nba_boxscoreplayertrackv3_1
   🔴 nba_boxscorescoringv3_0
   🔴 nba_boxscorescoringv3_1
   ... and 41 more tables

❌ Cleanup cancelled. No tables were deleted.
   Type 'DELETE' exactly to confirm deletion next time.

❌ Cleanup cancelled. No tables were deleted.
   Type 'DELETE' exactly to confirm deletion next time.


In [33]:
# 🗑️  AUTO-DELETE OLD NAMING CONVENTION TABLES (USER CONFIRMED)
import psycopg2
import json

print("🗑️  EXECUTING AUTOMATIC CLEANUP OF OLD NAMING CONVENTION TABLES")
print("="*75)

# Use the old_naming list we already have
print(f"📊 Deleting {len(old_naming)} tables with old naming convention")
print(f"✅ Keeping {len(new_naming)} tables with new meaningful naming")

# Show what we're deleting
print(f"\n🔴 TABLES BEING DELETED:")
for i, table in enumerate(old_naming, 1):
    if i <= 15:  # Show first 15
        print(f"   {i:2d}. {table}")
    elif i == 16:
        print(f"   ... and {len(old_naming) - 15} more tables")

print(f"\n🚀 Starting cleanup of {len(old_naming)} tables...")

# Connect to database
conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

deleted_count = 0
error_count = 0

for i, table in enumerate(old_naming, 1):
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table}")
        conn.commit()
        deleted_count += 1
        
        # Progress indicator every 10 tables
        if i % 10 == 0 or i == len(old_naming):
            print(f"   ✅ Progress: {i}/{len(old_naming)} ({i/len(old_naming)*100:.1f}%) - {table}")
            
    except Exception as e:
        print(f"   ❌ Error deleting {table}: {e}")
        error_count += 1
        continue

cursor.close()
conn.close()

print(f"\n🎉 DATABASE CLEANUP COMPLETE!")
print("="*50)
print(f"✅ Successfully deleted: {deleted_count} tables")
print(f"❌ Errors encountered: {error_count} tables")
print(f"🏁 Database now uses ONLY meaningful table names!")

if deleted_count > 0:
    print(f"\n📊 CLEANUP SUMMARY:")
    print(f"   BEFORE: {len(old_naming) + len(new_naming)} total tables ({len(old_naming)} old + {len(new_naming)} new)")
    print(f"   AFTER:  ~{len(new_naming)} total tables (all with meaningful names)")
    print(f"\n✨ All future endpoint processing will use meaningful names like:")
    print(f"   • nba_commonplayerinfo_commonplayerinfo")
    print(f"   • nba_boxscoreadvancedv3_playerstats")
    print(f"   • nba_boxscoreadvancedv3_teamstats")

🗑️  EXECUTING AUTOMATIC CLEANUP OF OLD NAMING CONVENTION TABLES
📊 Deleting 51 tables with old naming convention
✅ Keeping 52 tables with new meaningful naming

🔴 TABLES BEING DELETED:
    1. nba_boxscoreadvancedv3_0
    2. nba_boxscoreadvancedv3_1
    3. nba_boxscorefourfactorsv3_0
    4. nba_boxscorefourfactorsv3_1
    5. nba_boxscoremiscv3_0
    6. nba_boxscoremiscv3_1
    7. nba_boxscoreplayertrackv3_0
    8. nba_boxscoreplayertrackv3_1
    9. nba_boxscorescoringv3_0
   10. nba_boxscorescoringv3_1
   11. nba_boxscoresummaryv2_0
   12. nba_boxscoresummaryv2_1
   13. nba_boxscoresummaryv2_2
   14. nba_boxscoresummaryv2_3
   15. nba_boxscoresummaryv2_4
   ... and 36 more tables

🚀 Starting cleanup of 51 tables...
   ✅ Progress: 10/51 (19.6%) - nba_boxscorescoringv3_1
   ✅ Progress: 20/51 (39.2%) - nba_boxscoretraditionalv3_0
   ✅ Progress: 30/51 (58.8%) - nba_playbyplayv3_0
   ✅ Progress: 40/51 (78.4%) - nba_playerdashboardbyclutch_7
   ✅ Progress: 50/51 (98.0%) - nba_playerdashboardby

In [34]:
# ✅ VERIFY CLEANUP SUCCESS - CHECK REMAINING TABLES
import psycopg2
import json

print("✅ CLEANUP VERIFICATION")
print("="*40)

# Connect to database
conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

# Get all NBA endpoint tables after cleanup
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' 
    AND table_name LIKE 'nba_%'
    AND table_name NOT IN ('nba_games', 'nba_players', 'nba_teams')
    ORDER BY table_name
""")
remaining_tables = [row[0] for row in cursor.fetchall()]

# Check naming convention
old_format_remaining = [t for t in remaining_tables if t.split('_')[-1].isdigit()]
new_format_tables = [t for t in remaining_tables if not t.split('_')[-1].isdigit()]

print(f"📊 Total endpoint tables remaining: {len(remaining_tables)}")
print(f"🔴 Old naming format remaining: {len(old_format_remaining)}")
print(f"✅ New naming format: {len(new_format_tables)}")

if old_format_remaining:
    print(f"\n⚠️  OLD FORMAT TABLES STILL PRESENT:")
    for table in old_format_remaining:
        print(f"   {table}")
else:
    print(f"\n🎉 SUCCESS! No old naming convention tables remain!")

print(f"\n📋 SAMPLE NEW FORMAT TABLES:")
for i, table in enumerate(new_format_tables[:10], 1):
    print(f"   {i:2d}. {table}")
if len(new_format_tables) > 10:
    print(f"   ... and {len(new_format_tables) - 10} more tables")

# Show some specific examples
print(f"\n🏀 COMMONPLAYERINFO TABLES (should all be meaningful names):")
cpi_tables = [t for t in remaining_tables if 'commonplayerinfo' in t.lower()]
for table in cpi_tables:
    print(f"   ✅ {table}")

cursor.close()
conn.close()

print(f"\n🎯 DATABASE CLEANUP STATUS: {'✅ COMPLETE' if not old_format_remaining else '⚠️ INCOMPLETE'}")
print(f"🚀 All future NBA data collection will use meaningful table names!")

✅ CLEANUP VERIFICATION
📊 Total endpoint tables remaining: 53
🔴 Old naming format remaining: 1
✅ New naming format: 52

⚠️  OLD FORMAT TABLES STILL PRESENT:
   nba_commonplayerinfo_0

📋 SAMPLE NEW FORMAT TABLES:
    1. nba_boxscoreadvancedv3_playerstats
    2. nba_boxscoreadvancedv3_teamstats
    3. nba_boxscorefourfactorsv3_playerstats
    4. nba_boxscorefourfactorsv3_teamstats
    5. nba_boxscoremiscv3_playerstats
    6. nba_boxscoremiscv3_teamstats
    7. nba_boxscoreplayertrackv3_playerstats
    8. nba_boxscoreplayertrackv3_teamstats
    9. nba_boxscorescoringv3_playerstats
   10. nba_boxscorescoringv3_teamstats
   ... and 42 more tables

🏀 COMMONPLAYERINFO TABLES (should all be meaningful names):
   ✅ nba_commonplayerinfo_0
   ✅ nba_commonplayerinfo_availableseasons
   ✅ nba_commonplayerinfo_commonplayerinfo
   ✅ nba_commonplayerinfo_playerheadlinestats

🎯 DATABASE CLEANUP STATUS: ⚠️ INCOMPLETE
🚀 All future NBA data collection will use meaningful table names!


In [ ]:
# 🧹 CLEAN UP REMAINING OLD NAMING TABLE
import psycopg2
import json

print("🧹 FINAL CLEANUP - REMOVING LAST OLD NAMING TABLE")
print("="*50)

# The remaining old naming table
remaining_old_table = "nba_commonplayerinfo_0"

print(f"🔴 Found remaining old naming table: {remaining_old_table}")
print(f"✅ This table will be deleted to complete the cleanup")

# Connect to database
conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

try:
    # Check if table exists and get row count first
    cursor.execute(f"SELECT COUNT(*) FROM {remaining_old_table}")
    row_count = cursor.fetchone()[0]
    print(f"📊 Table {remaining_old_table} has {row_count:,} rows")
    
    # Delete the table
    print(f"🗑️  Deleting {remaining_old_table}...")
    cursor.execute(f"DROP TABLE IF EXISTS {remaining_old_table}")
    conn.commit()
    print(f"✅ Successfully deleted {remaining_old_table}")
    
except Exception as e:
    print(f"❌ Error deleting {remaining_old_table}: {e}")

# Final verification
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public' 
    AND table_name LIKE 'nba_%'
    AND table_name NOT IN ('nba_games', 'nba_players', 'nba_teams')
    ORDER BY table_name
""")
final_tables = [row[0] for row in cursor.fetchall()]

# Check final naming convention
final_old_format = [t for t in final_tables if t.split('_')[-1].isdigit()]
final_new_format = [t for t in final_tables if not t.split('_')[-1].isdigit()]

print(f"\n🎯 FINAL VERIFICATION:")
print(f"📊 Total endpoint tables: {len(final_tables)}")
print(f"🔴 Old naming format: {len(final_old_format)}")
print(f"✅ New naming format: {len(final_new_format)}")

if final_old_format:
    print(f"\n⚠️  REMAINING OLD FORMAT TABLES:")
    for table in final_old_format:
        print(f"   {table}")
else:
    print(f"\n🎉 PERFECT! All tables now use meaningful naming convention!")
    print(f"🏆 Database cleanup is 100% complete!")

cursor.close()
conn.close()

In [45]:
# 📋 SHOW ALL TABLES CURRENTLY IN DATABASE
import psycopg2
import json

print("📋 ALL TABLES IN DATABASE")
print("="*60)

# Connect to database
with open('../endpoints/config/database_config.json', 'r') as f:
    db_config = json.load(f)

conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

# Get all tables
cursor.execute("""
    SELECT table_name, 
           (SELECT COUNT(*) FROM information_schema.columns WHERE table_name = t.table_name) as column_count
    FROM information_schema.tables t
    WHERE table_schema = 'public' 
    ORDER BY table_name
""")
all_tables_info = cursor.fetchall()

# Categorize tables
master_tables = []
endpoint_tables = []
other_tables = []

for table_name, col_count in all_tables_info:
    if table_name in ['nba_games', 'nba_players', 'nba_teams']:
        master_tables.append((table_name, col_count))
    elif table_name.startswith('nba_'):
        endpoint_tables.append((table_name, col_count))
    else:
        other_tables.append((table_name, col_count))

print(f"📊 SUMMARY:")
print(f"   🏀 Master tables: {len(master_tables)}")
print(f"   🎯 Endpoint tables: {len(endpoint_tables)}")
print(f"   📝 Other tables: {len(other_tables)}")
print(f"   📋 Total tables: {len(all_tables_info)}")

# Show master tables with row counts
print(f"\n🏀 MASTER TABLES:")
for table_name, col_count in master_tables:
    try:
        if table_name == 'nba_games':
            cursor.execute(f"SELECT COUNT(DISTINCT gameid), COUNT(*) FROM {table_name}")
            unique_count, total_count = cursor.fetchone()
            print(f"   ✅ {table_name}: {unique_count:,} unique games, {total_count:,} total rows, {col_count} columns")
        elif table_name == 'nba_players':
            cursor.execute(f"SELECT COUNT(DISTINCT playerid), COUNT(*) FROM {table_name}")
            unique_count, total_count = cursor.fetchone()
            print(f"   ✅ {table_name}: {unique_count:,} unique players, {total_count:,} total rows, {col_count} columns")
        elif table_name == 'nba_teams':
            cursor.execute(f"SELECT COUNT(DISTINCT teamid), COUNT(*) FROM {table_name}")
            unique_count, total_count = cursor.fetchone()
            print(f"   ✅ {table_name}: {unique_count:,} unique teams, {total_count:,} total rows, {col_count} columns")
    except Exception as e:
        print(f"   ❌ {table_name}: Error reading table - {str(e)[:50]}...")

# Show endpoint tables
print(f"\n🎯 ENDPOINT TABLES ({len(endpoint_tables)} total):")

# Check naming convention
old_naming_tables = [t for t in endpoint_tables if t[0].split('_')[-1].isdigit()]
new_naming_tables = [t for t in endpoint_tables if not t[0].split('_')[-1].isdigit()]

print(f"   ✅ New naming convention: {len(new_naming_tables)} tables")
print(f"   🔴 Old naming convention: {len(old_naming_tables)} tables")

if old_naming_tables:
    print(f"\n   🔴 OLD NAMING CONVENTION TABLES:")
    for table_name, col_count in old_naming_tables:
        try:
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            row_count = cursor.fetchone()[0]
            print(f"      {table_name}: {row_count:,} rows, {col_count} columns")
        except:
            print(f"      {table_name}: Error reading, {col_count} columns")

# Show sample of new naming tables
print(f"\n   ✅ NEW NAMING CONVENTION TABLES (first 15):")
for i, (table_name, col_count) in enumerate(new_naming_tables[:15], 1):
    try:
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]
        print(f"   {i:2d}. {table_name}: {row_count:,} rows, {col_count} columns")
    except:
        print(f"   {i:2d}. {table_name}: Error reading, {col_count} columns")

if len(new_naming_tables) > 15:
    print(f"      ... and {len(new_naming_tables) - 15} more endpoint tables")

# Show other tables
if other_tables:
    print(f"\n📝 OTHER TABLES:")
    for table_name, col_count in other_tables:
        try:
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            row_count = cursor.fetchone()[0]
            print(f"   📋 {table_name}: {row_count:,} rows, {col_count} columns")
        except:
            print(f"   📋 {table_name}: Error reading, {col_count} columns")

cursor.close()
conn.close()

print(f"\n{'='*60}")
print(f"🎯 DATABASE STATUS: {'✅ Clean (all meaningful names)' if not old_naming_tables else f'⚠️ {len(old_naming_tables)} old naming tables remain'}")

📋 ALL TABLES IN DATABASE
📊 SUMMARY:
   🏀 Master tables: 3
   🎯 Endpoint tables: 75
   📝 Other tables: 7
   📋 Total tables: 85

🏀 MASTER TABLES:
📊 SUMMARY:
   🏀 Master tables: 3
   🎯 Endpoint tables: 75
   📝 Other tables: 7
   📋 Total tables: 85

🏀 MASTER TABLES:
   ✅ nba_games: 52,782 unique games, 52,782 total rows, 16 columns
   ✅ nba_players: 571 unique players, 571 total rows, 19 columns
   ✅ nba_teams: 30 unique teams, 30 total rows, 11 columns

🎯 ENDPOINT TABLES (75 total):
   ✅ New naming convention: 52 tables
   🔴 Old naming convention: 23 tables

   🔴 OLD NAMING CONVENTION TABLES:
      nba_boxscorefourfactorsv3_0: 994 rows, 23 columns
      nba_boxscorefourfactorsv3_1: 76 rows, 15 columns
      nba_boxscoremiscv3_0: 948 rows, 27 columns
   ✅ nba_games: 52,782 unique games, 52,782 total rows, 16 columns
   ✅ nba_players: 571 unique players, 571 total rows, 19 columns
   ✅ nba_teams: 30 unique teams, 30 total rows, 11 columns

🎯 ENDPOINT TABLES (75 total):
   ✅ New naming conve

In [43]:
# 📋 CONCISE TABLE SUMMARY
import psycopg2
import json

print("📋 DATABASE TABLES SUMMARY")
print("="*45)

# Connect to database
with open('../endpoints/config/database_config.json', 'r') as f:
    db_config = json.load(f)

conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

# Get all tables
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' ORDER BY table_name")
all_tables = [row[0] for row in cursor.fetchall()]

# Categorize
master_tables = [t for t in all_tables if t in ['nba_games', 'nba_players', 'nba_teams']]
endpoint_tables = [t for t in all_tables if t.startswith('nba_') and t not in master_tables]
other_tables = [t for t in all_tables if not t.startswith('nba_')]

# Check naming conventions
old_naming = [t for t in endpoint_tables if t.split('_')[-1].isdigit()]
new_naming = [t for t in endpoint_tables if not t.split('_')[-1].isdigit()]

print(f"📊 TOTALS:")
print(f"   🏀 Master tables: {len(master_tables)}")
print(f"   🎯 Endpoint tables: {len(endpoint_tables)}")
print(f"      ✅ New naming: {len(new_naming)}")
print(f"      🔴 Old naming: {len(old_naming)}")
print(f"   📝 Other tables: {len(other_tables)}")
print(f"   📋 Grand total: {len(all_tables)}")

print(f"\n🏀 MASTER TABLES:")
for table in master_tables:
    print(f"   {table}")

if old_naming:
    print(f"\n🔴 OLD NAMING TABLES ({len(old_naming)}):")
    for table in old_naming:
        print(f"   {table}")

print(f"\n✅ NEW NAMING SAMPLE (first 10 of {len(new_naming)}):")
for i, table in enumerate(new_naming[:10], 1):
    print(f"   {i:2d}. {table}")
if len(new_naming) > 10:
    print(f"       ... and {len(new_naming) - 10} more")

if other_tables:
    print(f"\n📝 OTHER TABLES:")
    for table in other_tables:
        print(f"   {table}")

cursor.close()
conn.close()

print(f"\n🎯 STATUS: {'✅ All meaningful names' if not old_naming else f'⚠️ {len(old_naming)} old naming tables remain'}")

📋 DATABASE TABLES SUMMARY
📊 TOTALS:
   🏀 Master tables: 3
   🎯 Endpoint tables: 52
      ✅ New naming: 52
      🔴 Old naming: 0
   📝 Other tables: 7
   📋 Grand total: 62

🏀 MASTER TABLES:
   nba_games
   nba_players
   nba_teams

✅ NEW NAMING SAMPLE (first 10 of 52):
    1. nba_boxscoreadvancedv3_playerstats
    2. nba_boxscoreadvancedv3_teamstats
    3. nba_boxscorefourfactorsv3_playerstats
    4. nba_boxscorefourfactorsv3_teamstats
    5. nba_boxscoremiscv3_playerstats
    6. nba_boxscoremiscv3_teamstats
    7. nba_boxscoreplayertrackv3_playerstats
    8. nba_boxscoreplayertrackv3_teamstats
    9. nba_boxscorescoringv3_playerstats
   10. nba_boxscorescoringv3_teamstats
       ... and 42 more

📝 OTHER TABLES:
   failed_api_calls
   gleague_games
   gleague_players
   gleague_teams
   wnba_games
   wnba_players
   wnba_teams

🎯 STATUS: ✅ All meaningful names
📊 TOTALS:
   🏀 Master tables: 3
   🎯 Endpoint tables: 52
      ✅ New naming: 52
      🔴 Old naming: 0
   📝 Other tables: 7
   📋 G

In [42]:
# 🗑️  DELETE ALL REMAINING OLD NAMING CONVENTION TABLES
import psycopg2
import json

print("🗑️  FINAL CLEANUP - REMOVING ALL REMAINING OLD NAMING TABLES")
print("="*70)

# Use the old_naming list from the previous cell
print(f"🔴 Found {len(old_naming)} tables with old naming convention")
print(f"✅ Will keep {len(new_naming)} tables with meaningful names")

print(f"\n📋 TABLES TO BE DELETED:")
for i, table in enumerate(old_naming, 1):
    print(f"   {i:2d}. {table}")

print(f"\n🚀 Starting automatic cleanup...")

# Connect to database
conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

deleted_count = 0
error_count = 0

for i, table in enumerate(old_naming, 1):
    try:
        print(f"   🗑️  [{i:2d}/{len(old_naming)}] Deleting {table}")
        cursor.execute(f"DROP TABLE IF EXISTS {table}")
        conn.commit()
        deleted_count += 1
        
    except Exception as e:
        print(f"   ❌ Error deleting {table}: {e}")
        error_count += 1
        continue

cursor.close()
conn.close()

print(f"\n🎉 FINAL CLEANUP COMPLETE!")
print("="*50)
print(f"✅ Successfully deleted: {deleted_count} tables")
print(f"❌ Errors encountered: {error_count} tables")

if deleted_count == len(old_naming):
    print(f"🏆 PERFECT! All old naming convention tables removed!")
    print(f"🎯 Database now has ONLY meaningful table names!")
else:
    print(f"⚠️  Some tables may still remain - check final verification below")

print(f"\n📊 CLEANUP SUMMARY:")
print(f"   BEFORE: {len(old_naming) + len(new_naming)} endpoint tables ({len(old_naming)} old + {len(new_naming)} new)")
print(f"   AFTER:  {len(new_naming)} endpoint tables (all meaningful names)")

🗑️  FINAL CLEANUP - REMOVING ALL REMAINING OLD NAMING TABLES
🔴 Found 23 tables with old naming convention
✅ Will keep 52 tables with meaningful names

📋 TABLES TO BE DELETED:
    1. nba_boxscorefourfactorsv3_0
    2. nba_boxscorefourfactorsv3_1
    3. nba_boxscoremiscv3_0
    4. nba_boxscoremiscv3_1
    5. nba_boxscoresummaryv2_0
    6. nba_boxscoresummaryv2_1
    7. nba_boxscoresummaryv2_2
    8. nba_boxscoresummaryv2_3
    9. nba_boxscoresummaryv2_4
   10. nba_boxscoresummaryv2_5
   11. nba_boxscoresummaryv2_6
   12. nba_boxscoresummaryv2_7
   13. nba_boxscoresummaryv2_8
   14. nba_boxscoretraditionalv3_0
   15. nba_boxscoretraditionalv3_1
   16. nba_boxscoretraditionalv3_2
   17. nba_boxscoreusagev3_0
   18. nba_boxscoreusagev3_1
   19. nba_commonplayerinfo_0
   20. nba_commonplayerinfo_1
   21. nba_commonplayerinfo_2
   22. nba_playbyplayv3_0
   23. nba_playbyplayv3_1

🚀 Starting automatic cleanup...
   🗑️  [ 1/23] Deleting nba_boxscorefourfactorsv3_0
   🗑️  [ 2/23] Deleting nba_bo

In [38]:
# ✅ FINAL VERIFICATION - CHECK DATABASE IS CLEAN
import psycopg2
import json

print("✅ FINAL VERIFICATION - DATABASE CLEANUP STATUS")
print("="*55)

# Connect to database
conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

# Get all current tables
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' ORDER BY table_name")
current_tables = [row[0] for row in cursor.fetchall()]

# Categorize current tables
master_tables = [t for t in current_tables if t in ['nba_games', 'nba_players', 'nba_teams']]
endpoint_tables = [t for t in current_tables if t.startswith('nba_') and t not in master_tables]
other_tables = [t for t in current_tables if not t.startswith('nba_')]

# Check naming conventions
remaining_old_naming = [t for t in endpoint_tables if t.split('_')[-1].isdigit()]
clean_new_naming = [t for t in endpoint_tables if not t.split('_')[-1].isdigit()]

print(f"📊 FINAL DATABASE STATE:")
print(f"   🏀 Master tables: {len(master_tables)}")
print(f"   🎯 Endpoint tables: {len(endpoint_tables)}")
print(f"      ✅ Meaningful names: {len(clean_new_naming)}")
print(f"      🔴 Old naming: {len(remaining_old_naming)}")
print(f"   📝 Other tables: {len(other_tables)}")
print(f"   📋 Total tables: {len(current_tables)}")

if remaining_old_naming:
    print(f"\n⚠️  REMAINING OLD NAMING TABLES:")
    for table in remaining_old_naming:
        print(f"   🔴 {table}")
    print(f"\n❌ CLEANUP STATUS: INCOMPLETE - {len(remaining_old_naming)} tables remain")
else:
    print(f"\n🎉 CLEANUP STATUS: 100% COMPLETE!")
    print(f"🏆 ALL endpoint tables now use meaningful names!")

print(f"\n📋 SAMPLE MEANINGFUL TABLE NAMES:")
for i, table in enumerate(clean_new_naming[:8], 1):
    print(f"   {i}. {table}")
if len(clean_new_naming) > 8:
    print(f"   ... and {len(clean_new_naming) - 8} more meaningful tables")

cursor.close()
conn.close()

print(f"\n{'='*55}")
if not remaining_old_naming:
    print(f"🎯 SUCCESS: Database is now 100% clean!")
    print(f"🚀 All future NBA data collection uses meaningful names!")
    print(f"✨ Examples: nba_commonplayerinfo_commonplayerinfo")
    print(f"✨          nba_boxscoreadvancedv3_playerstats")
else:
    print(f"⚠️  {len(remaining_old_naming)} old naming tables still need cleanup")

✅ FINAL VERIFICATION - DATABASE CLEANUP STATUS
📊 FINAL DATABASE STATE:
   🏀 Master tables: 3
   🎯 Endpoint tables: 56
      ✅ Meaningful names: 52
      🔴 Old naming: 4
   📝 Other tables: 7
   📋 Total tables: 66

⚠️  REMAINING OLD NAMING TABLES:
   🔴 nba_boxscoremiscv3_0
   🔴 nba_boxscoremiscv3_1
   🔴 nba_boxscoreusagev3_0
   🔴 nba_boxscoreusagev3_1

❌ CLEANUP STATUS: INCOMPLETE - 4 tables remain

📋 SAMPLE MEANINGFUL TABLE NAMES:
   1. nba_boxscoreadvancedv3_playerstats
   2. nba_boxscoreadvancedv3_teamstats
   3. nba_boxscorefourfactorsv3_playerstats
   4. nba_boxscorefourfactorsv3_teamstats
   5. nba_boxscoremiscv3_playerstats
   6. nba_boxscoremiscv3_teamstats
   7. nba_boxscoreplayertrackv3_playerstats
   8. nba_boxscoreplayertrackv3_teamstats
   ... and 44 more meaningful tables

⚠️  4 old naming tables still need cleanup


In [39]:
# 🧹 CLEAN UP THE FINAL 4 OLD NAMING TABLES
import psycopg2
import json

print("🧹 FINAL CLEANUP - LAST 4 OLD NAMING TABLES")
print("="*50)

# The remaining tables from verification above
final_old_tables = [
    'nba_boxscoremiscv3_0',
    'nba_boxscoremiscv3_1', 
    'nba_boxscoreusagev3_0',
    'nba_boxscoreusagev3_1'
]

print(f"🔴 Found {len(final_old_tables)} remaining old naming tables:")
for i, table in enumerate(final_old_tables, 1):
    print(f"   {i}. {table}")

print(f"\n🚀 Deleting final old naming tables...")

# Connect to database
conn = psycopg2.connect(
    host=db_config['host'],
    database=db_config['name'],
    user=db_config['user'],
    password=db_config['password'],
    port=db_config['port']
)
cursor = conn.cursor()

final_deleted = 0
final_errors = 0

for i, table in enumerate(final_old_tables, 1):
    try:
        print(f"   🗑️  [{i}/{len(final_old_tables)}] Deleting {table}")
        cursor.execute(f"DROP TABLE IF EXISTS {table}")
        conn.commit()
        final_deleted += 1
        print(f"   ✅ Successfully deleted {table}")
        
    except Exception as e:
        print(f"   ❌ Error deleting {table}: {e}")
        final_errors += 1

# Final check
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_name LIKE 'nba_%' ORDER BY table_name")
final_check_tables = [row[0] for row in cursor.fetchall()]

master_final = [t for t in final_check_tables if t in ['nba_games', 'nba_players', 'nba_teams']]
endpoint_final = [t for t in final_check_tables if t.startswith('nba_') and t not in master_final]
last_old_check = [t for t in endpoint_final if t.split('_')[-1].isdigit()]
final_meaningful = [t for t in endpoint_final if not t.split('_')[-1].isdigit()]

cursor.close()
conn.close()

print(f"\n🎉 FINAL CLEANUP RESULTS:")
print(f"   ✅ Deleted: {final_deleted}/{len(final_old_tables)} tables")
print(f"   ❌ Errors: {final_errors} tables")

print(f"\n📊 FINAL DATABASE STATE:")
print(f"   🏀 Master tables: {len(master_final)}")
print(f"   🎯 Endpoint tables: {len(endpoint_final)}")
print(f"      ✅ Meaningful names: {len(final_meaningful)}")
print(f"      🔴 Old naming: {len(last_old_check)}")

if not last_old_check:
    print(f"\n🏆 PERFECT! DATABASE IS 100% CLEAN!")
    print(f"✨ All {len(final_meaningful)} endpoint tables use meaningful names!")
    print(f"🎯 Mission accomplished!")
else:
    print(f"\n⚠️  {len(last_old_check)} old naming tables still remain:")
    for table in last_old_check:
        print(f"   🔴 {table}")

🧹 FINAL CLEANUP - LAST 4 OLD NAMING TABLES
🔴 Found 4 remaining old naming tables:
   1. nba_boxscoremiscv3_0
   2. nba_boxscoremiscv3_1
   3. nba_boxscoreusagev3_0
   4. nba_boxscoreusagev3_1

🚀 Deleting final old naming tables...
   🗑️  [1/4] Deleting nba_boxscoremiscv3_0
   ✅ Successfully deleted nba_boxscoremiscv3_0
   🗑️  [2/4] Deleting nba_boxscoremiscv3_1
   ✅ Successfully deleted nba_boxscoremiscv3_1
   🗑️  [3/4] Deleting nba_boxscoreusagev3_0
   ✅ Successfully deleted nba_boxscoreusagev3_0
   🗑️  [4/4] Deleting nba_boxscoreusagev3_1
   ✅ Successfully deleted nba_boxscoreusagev3_1

🎉 FINAL CLEANUP RESULTS:
   ✅ Deleted: 4/4 tables
   ❌ Errors: 0 tables

📊 FINAL DATABASE STATE:
   🏀 Master tables: 3
   🎯 Endpoint tables: 52
      ✅ Meaningful names: 52
      🔴 Old naming: 0

🏆 PERFECT! DATABASE IS 100% CLEAN!
✨ All 52 endpoint tables use meaningful names!
🎯 Mission accomplished!
